# Data import

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
listings = pd.read_csv('../data/data_jan_listings_3plus.csv',low_memory=False)

In [3]:
def str_to_price(strn):
    '''The function converts a price entry from string to float and removes the $ character'''
    
    if type(strn)== str:
        return float(strn.strip('$').replace(',',''))
    return strn


def price_tonumerical(df, price_columns):
    '''This function takes as input a dataframe and a list of price column names and converts the columnns to
    floats'''
    
    for column in price_columns:
        df[[column]] = df[[column]].applymap(str_to_price)
    return df

def calculate_cleaning_ratio(df):
    '''This function calculates the ratio of cleaning fee to the nightly price'''
    
    df['cleaning_ratio']=round(df['cleaning_fee']/df['price'],2)
    return df

def calculate_deposit_ratio(df):
    '''This function calculates the ratio of security deposit fee to the nightly price'''
    
    df['deposit_ratio']=round(df['security_deposit']/df['price'],2)
    return df


price_columns=['price', 'security_deposit', 'cleaning_fee', 'extra_people']
price_tonumerical(listings, price_columns)
calculate_cleaning_ratio(listings)


,id,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,host_since,host_location,host_about,host_response_time,host_response_rate,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_identity_verified,street,neighbourhood_cleansed,zipcode,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month,cleaning_ratio
0,11551.0,Arty and Bright London Apartment in Zone 2,Unlike most rental apartments out there my fla...,"Amenities Bedding: 1 Double bed, 1 living room...",Unlike most rental apartments out there my fla...,family,Not even 10 minutes by metro from Victoria Sta...,No Smoking (very strict) Check-in time is afte...,Tons of buses (24hrs) go into central London f...,Guest will have access to the entire apartment,No interaction with guests as you book the ent...,No Smoking (very strict) No pets are allowed i...,2009-10-03,"London, England, United Kingdom","Hello, I'm a friendly Italian man with a very ...",within an hour,100%,Brixton,3.0,3.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,"London, United Kingdom",Lambeth,SW9 8DG,51.46225,-0.11732000000000001,t,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,Kitchen,""Paid par...",88.0,$645.00,"$2,350.00",400.0,30.0,2.0,25.0,2.0,180.0,17.0,42,72,344,186,3.0,2010-03-21,2019-11-17,92.0,9.0,9.0,10.0,10.0,9.0,9.0,t,strict_14_with_grace_period,f,t,1.56,0.34
1,13913.0,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,"Hello Everyone, I'm offering my lovely double ...",My bright double bedroom with a large window h...,business,Finsbury Park is a friendly melting pot commun...,For art lovers I can give guest my Tate Member...,The flat only a 10 minute walk to Finsbury Par...,Guest will have access to the self catering ki...,I like to have little chats with my guest over...,I'm an artist and have my artwork up on the wa...,2009-11-16,"London, England, United Kingdom",I am a Multi-Media Visual Artist and Creative ...,within a few hours,70%,LB of Islington,4.0,4.0,"['email', 'phone', 'facebook', 'reviews', 'off...",t,"Islington, Greater London, United Kingdom",Islington,N4 3,51.568020000000004,-0.11120999999999999,t,Apartment,Private room,2.0,1.0,1.0,0.0,Real Bed,"{TV,""Cable TV"",Wifi,Kitchen,""Paid parking off ...",65.0,$333.00,"$1,176.00",100.0,15.0,1.0,15.0,1.0,29.0,30.0,60,90,365,20,6.0,2010-08-18,2019-11-25,97.0,10.0,10.0,10.0,10.0,9.0,9.0,f,moderate,f,f,0.17,0.23
2,15400.0,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Bright Chelsea Apartment This is a bright one...,Lots of windows and light. St Luke's Gardens ...,romantic,It is Chelsea.,The building next door is in the process of be...,The underground stations are South Kensington ...,There are two wardrobes for guests exclusive u...,If I am in the country I like to welcome my gu...,NO SMOKING PLEASE.. No unauthorised guests. No...,2009-12-05,"Kensington, England, United Kingdom","English, grandmother, I have travelled quite ...",within a few hours,100%,Chelsea,1.0,1.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,"London, United Kingdom",Kensington and Chelsea,SW3,51.48796,-0.16898,t,Apartment,Entire home/apt,2.0,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",100.0,$600.00,"$2,250.00",150.0,50.0,2.0,0.

# Clusters database building

In [30]:
def price_cat(x,pctl):
    '''Transforms an absolute price into a price category, given a list of percentiles'''
    if x <pctl[1]:
        return 'very_cheap'
    elif x >=pctl[1] and x <pctl[3]:
        return 'cheap'
    elif x >=pctl[3] and x <pctl[5]:
        return 'average'
    elif x >=pctl[5] and x <pctl[7]:
        return 'expensive'
    return 'very_expensive'

def clustering(data):
    '''Returns a dataframe containing the relevant clustering variables'''
    #pctl=[]
    columns=['location', 'price-boroughwise', 'property_type', 'review_scores_rating']
    cluster_df = pd.DataFrame(columns=columns)
    #for p in range(1,10):
    #    pctl.append(np.nanpercentile(data['price'], 10*p))
    #
    #cluster_df['price-citywise']=data['price'].apply(lambda x:price_cat(x,pctl))
    percentiles=dict()
    neighbourhoods = listings['neighbourhood_cleansed'].unique()
    for neigh in neighbourhoods:
        neigh_listings = listings[listings['neighbourhood_cleansed'] == neigh]
        indices= neigh_listings.index.values.tolist()
        pctl=[]
        for p in range(1,10):
            pctl.append(np.nanpercentile(neigh_listings['price'], 10*p))
        #neigh_listings['price'].apply(lambda x:price_cat(x,pctl))
        percentiles[neigh] = pctl
        for index in indices:
            cluster_df.loc[index,'price-boroughwise']=price_cat(neigh_listings.loc[index,'price'],pctl)
            
    
    bedroom_convert = {0.0: 'small',
                  1.0: 'small',
                  2.0: 'large',
                  3.0: 'large', 4.0: 'large',5.0: 'large',6.0: 'large',7.0: 'large',
                   8.0: 'large',9.0: 'large',10.0: 'large',11.0: 'large',
                   16.0: 'large',22.0: 'large'}
    cluster_df['property_type'] =  data['bedrooms'].map(bedroom_convert)
    cluster_df['room/apt']=data[['room_type']].applymap(lambda x: 'apartment' if x == 'Entire home/apt' else 'room')# if x == 'Private room' else 'Other'))
    
    cluster_df['location']=data['neighbourhood_cleansed']
    
    cluster_df['review_scores_rating']=data['review_scores_rating']
    
    cluster_df['property_type'] = cluster_df.apply(lambda x: 'room' if x['room/apt']== 'room' else x['property_type'], axis=1)
    #cluster_df['listing_id'] = data['id']
    cluster_df=cluster_df.drop(columns='room/apt')
    matching_table = clusters.copy()
    matching_table = matching_table.groupby(['location','price-boroughwise','property_type']).count().reset_index().drop(columns='review_scores_rating')
    matching_table['cluster_id'] = matching_table.index
    return cluster_df, matching_table, percentiles

clusters, matching_table, percentiles=clustering(listings)


/Users/lcavicchia/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1368: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [31]:
clusters.to_csv('clusters.csv')
matching_table.to_csv('clusters_table.csv')

In [14]:
clusters

,location,price-boroughwise,property_type,review_scores_rating,listing_id
0,Lambeth,expensive,small,92.0,11551.0
33,Lambeth,cheap,room,98.0,53242.0
34,Lambeth,expensive,small,96.0,53245.0
50,Lambeth,cheap,room,96.0,72913.0
51,Lambeth,cheap,large,98.0,73431.0
...,...,...,...,...,...
51047,Sutton,expensive,small,100.0,39963193.0
51074,Sutton,expensive,small,98.0,39988769.0
51098,Sutton,very_expensive,small,40.0,40019636.0
51255,Sutton,expensive,large,60.0,40147104.0


In [7]:
def user_ranking(location, price, size, rating, clusters):
    
    
    price_c=price_cat(price,percentiles[location])
    
    
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False)
    cluster_average = cluster['review_scores_rating'].mean()
    user_rank =int(cluster[cluster['review_scores_rating']== rating].iloc[0].loc['ranking'])
    return user_rank, cluster_average, cluster.shape[0]
    

user_rank, mean_rating, cluster_size=user_ranking('Hackney', 120, 'large', 95, clusters)

print(f"Out of {cluster_size} similar properties with an average rating of {round(mean_rating/20,2)}, your listing \
is ranked n. {user_rank}")

Out of 641 similar properties with an average rating of 4.72, your listing is ranked n. 333


In [8]:
user_rank, mean_rating, cluster_size

(333, 94.41341653666147, 641)

In [24]:
def top_rated(location, price, size, clusters, top=10):
    
    price_c=price_cat(price,percentiles[location])
    
    
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    cluster['ranking'] = cluster['review_scores_rating'].rank(method='min',ascending=False)
    
    sorted_cluster = cluster.sort_values(by='ranking')
    
    nrows= round(cluster.shape[0]/ top)
    
    return sorted_cluster.iloc[:nrows].index.values.tolist()

In [25]:
top = top_rated('Hackney', 120, 'large', clusters)

In [13]:
def cluster_selection(location, price, size, clusters):
    price_c=price_cat(price,percentiles[location])
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    return cluster
cluster_selection('Hackney', 120, 'large', clusters)

,location,price-boroughwise,property_type,review_scores_rating,listing_id
107,Hackney,very_expensive,large,90.0,102391.0
116,Hackney,very_expensive,large,60.0,109299.0
208,Hackney,very_expensive,large,98.0,243077.0
211,Hackney,very_expensive,large,91.0,244125.0
376,Hackney,very_expensive,large,93.0,359974.0
388,Hackney,very_expensive,large,100.0,367181.0
408,Hackney,very_expensive,large,90.0,379137.0
424,Hackney,very_expensive,large,79.0,386969.0
441,Hackney,very_expensive,large,95.0,388721.0
454,Hackney,very_expensive,large,96.0,390041.0


In [32]:
def cluster_mapping(location, price, size, clusters, listings):
    
    price_c=price_cat(price,percentiles[location])
    cluster = clusters[(clusters['location'] == location ) &  (clusters['price-boroughwise'] == price_c ) &\
              (clusters['property_type'] == size )\
            ].copy()
    
    indices = cluster.index.values.tolist()
    coordinates = listings.iloc[indices][['latitude','longitude']]
    return coordinates

In [33]:
cluster_mapping('Hackney', 120, 'large', clusters,listings)

,latitude,longitude
107,51.54647,-0.056639999999999996
116,51.54599,-0.07426
208,51.553059999999995,-0.04562
211,51.5283,-0.08035
376,51.52795,-0.08272
388,51.565979999999996,-0.08578
408,51.55514,-0.05649
424,51.53,-0.08895
441,51.56054,-0.07686
454,51.54719,-0.06462000000000001


In [21]:
matching_table  = clusters.drop(columns=['listing_id']).copy()

In [17]:
matching_table

,location,price-boroughwise,property_type
0,Lambeth,expensive,small
33,Lambeth,cheap,room
34,Lambeth,expensive,small
50,Lambeth,cheap,room
51,Lambeth,cheap,large
...,...,...,...
51047,Sutton,expensive,small
51074,Sutton,expensive,small
51098,Sutton,very_expensive,small
51255,Sutton,expensive,large


In [27]:
matching_table = matching_table.groupby(['location','price-boroughwise','property_type']).count().reset_index().drop(columns='review_scores_rating')
matching_table['cluster_id']=matching_table.index

In [28]:
matching_table

,location,price-boroughwise,property_type,cluster_id
0,Barking and Dagenham,average,room,0
1,Barking and Dagenham,average,small,1
2,Barking and Dagenham,cheap,room,2
3,Barking and Dagenham,cheap,small,3
4,Barking and Dagenham,expensive,large,4
5,Barking and Dagenham,expensive,room,5
6,Barking and Dagenham,expensive,small,6
7,Barking and Dagenham,very_cheap,room,7
8,Barking and Dagenham,very_expensive,large,8
9,Barking and Dagenham,very_expensive,room,9


In [29]:
matching_table.to_csv('clusters_table.csv')